In [125]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [126]:
cred = pd.read_csv("credit_activities.csv")
proj = pd.read_csv("project_info.csv")
#prev = pd.read_csv("../RawData.csv")

/var/folders/gn/x53b3rps7jz6jbl5j86y21gr0000gn/T/ipykernel_45582/1767492222.py:1: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  cred = pd.read_csv("credit_activities.csv")


In [127]:
#prev = prev[['Project ID', 'Scope', ' Type', 'Reduction / Removal']]
#prev.columns = ['Project ID', 'Scope', 'Type', 'Reduction/Removal']

In [128]:
proj.columns

Index(['project_id', 'project_name', 'voluntary_registry', 'arb_wa_project',
       'voluntary_status', 'scope', 'type', 'type_rule', 'type_ml',
       'confidence', 'reduction_removal', 'methodology', 'methodology_version',
       'region', 'country', 'state', 'project_site_location',
       'project_developer', 'project_owner', 'offset_project_operator',
       'authorized_project_designee', 'verifier',
       'estimated_annual_emission_reductions', 'arb_id', 'arb_project_type',
       'arb_project_status', 'wa_id', 'wa_project_type', 'poa_id',
       'project_listed', 'project_registered', 'sustainability_certification',
       'project_type_from_the_registry', 'registry_documents',
       'project_website', 'notes_from_registry', 'notes_from_bctp',
       'project_description', 'added_to_database_version'],
      dtype='object')

In [129]:
proj_names = ['Project ID', 'Project Name', 'Voluntary Registry', 'ARB/WA Project', 
              'Voluntary Status', 'Scope', 'Type', 'Type Rule', 'Type ML',
              'Confidence Level', 'Reduction/Removal', 'Methodology/Protocol', 'Methodology Version', 
              'Region', 'Country', 'State', 'Project Site Location',
              'Project Developer', 'Project Owner', 'Offset Project Operator', 
              'Authorized Project Designee', 'Verifier',
              'Estimated Annual Emission Reductions', 'ARB ID', 'ARB Project Type',
              'ARB Project Status', 'WA ID', 'WA Project Type', 'POA ID', 
              'Project Listed', 'Project Registered', 'Sustainability Certification',
              'Project Type From the Registry', 'Registry Documents',
              'Project Website', 'Notes From Registry', 'Notes from BCTP', 
              'Project Description', 'Added to DB Version']

In [130]:
proj.columns = proj_names
proj1 = proj.iloc[:, :19]
proj2 = proj.drop(proj.columns[1:19], axis=1)

In [131]:
proj2.columns

Index(['Project ID', 'Offset Project Operator', 'Authorized Project Designee',
       'Verifier', 'Estimated Annual Emission Reductions', 'ARB ID',
       'ARB Project Type', 'ARB Project Status', 'WA ID', 'WA Project Type',
       'POA ID', 'Project Listed', 'Project Registered',
       'Sustainability Certification', 'Project Type From the Registry',
       'Registry Documents', 'Project Website', 'Notes From Registry',
       'Notes from BCTP', 'Project Description', 'Added to DB Version'],
      dtype='object')

In [132]:
cred[['vintage_year','issuance_date']] = cred[['vintage_year','issuance_date']].astype(str)
cred[['vintage_year','issuance_date','amount']] = cred[['vintage_year','issuance_date','amount']].fillna(0)
cred[['vintage_year','issuance_date']] = cred[['vintage_year','issuance_date']].replace('NaT', 0)
cred[['vintage_year','issuance_date']] = cred[['vintage_year','issuance_date']].replace('nan', 0)
cred[['vintage_year','issuance_date']] = cred[['vintage_year','issuance_date']].astype(str)
cred['vintage_year'] = cred['vintage_year'].str.replace('.0', '')
cred['temp1'] = cred['issuance_date'].str.contains('-')
cred['temp2'] = cred['issuance_date'].str.contains('/')
cred.loc[cred['temp1'], 'issuance_date'] = cred.loc[cred['temp1'], 'issuance_date'].str[:4]
cred.loc[cred['temp2'], 'issuance_date'] = cred.loc[cred['temp2'], 'issuance_date'].str.split('/').str[2].str[:4]

cred = cred.drop(columns=['temp1', 'temp2'])

In [133]:
cred.issuance_date.unique()

array(['2013', '2014', '2015', '2016', '2017', '2023', '2021', '2018',
       '2022', '2020', '2019', '2024', '2012', '2011', '2010', '2009',
       '2008', '2007', '2006', '2005', '2004', '2002', '0'], dtype=object)

In [134]:
cred.transaction_type.unique()

array(['issuance', 'buffer_contribution', 'retirement_cancellation',
       'buffer_subtraction', 'per'], dtype=object)

In [135]:
cred.sub_transaction_type.unique()

array(['issuance', 'buffer_contribution', 'retired_for_arb_compliance',
       'retired_for_quebec_compliance', 'retired_voluntarily',
       'cancelled_for_intentional_reversal', 'transfer_to_arb',
       'transfer_to_wa', 'cancelled_during_transfer_to_arb', 'retirement',
       'cancellation', 'buffer_pool_transfer_to_arb',
       'buffer_pool_cancelled_during_transfer_to_arb',
       'buffer_subtraction', 'per_issuance', 'buffer_release_to_issuance',
       'unintentional_reversal'], dtype=object)

In [136]:
cred.head()

,project_id,transaction_type,sub_transaction_type,amount,date,vintage_year,issuance_date,corsia_eligible,vintage_start,vintage_end
0,CAR992,issuance,issuance,107542.0,2013-10-22 00:00:00,2013,2013,NaN,NaN,NaN
1,ACR170,issuance,issuance,306517.0,2013-09-23 00:00:00,2012,2013,NaN,NaN,NaN
2,CAR973,issuance,issuance,1748472.0,2014-06-10 00:00:00,2013,2014,NaN,NaN,NaN
3,CAR973,issuance,issuance,119314.0,2015-09-08 00:00:00,2014,2015,NaN,NaN,NaN
4,CAR973,issuance,issuance,282627.0,2016-09-27 00:00:00,2015,2016,NaN,NaN,NaN


In [137]:
cred.issuance_date.value_counts()

issuance_date
2021    74325
2015    45378
2020    40853
2019    33841
2022    25206
2018    24977
2014    21363
2017    19454
2023    19452
2016    17876
2012    14246
2013    12628
0        9659
2011     8331
2009     6785
2010     5555
2024     1353
2008      606
2005      560
2007      510
2006       91
2004        3
2002        1
Name: count, dtype: int64

In [146]:
cred_group_issuanceDate = cred.groupby(['project_id', 'transaction_type', 'issuance_date']).amount.sum().reset_index()
cred_group_vintageDate = cred.groupby(['project_id', 'transaction_type', 'vintage_year']).amount.sum().reset_index()

In [147]:
cred_piv_iss = cred_group_issuanceDate.pivot(index=['project_id', 'issuance_date'], 
                      columns='transaction_type', values='amount').reset_index()
cred_piv_vin = cred_group_vintageDate.pivot(index=['project_id', 'vintage_year'], 
                      columns='transaction_type', values='amount').reset_index()

In [148]:
cred_piv_vin.head()

transaction_type,project_id,vintage_year,buffer_contribution,buffer_subtraction,issuance,per,retirement_cancellation
0,ACR102,2003,NaN,NaN,1898382.0,NaN,7666.0
1,ACR102,2004,NaN,NaN,3004613.0,NaN,NaN
2,ACR102,2005,NaN,NaN,3081011.0,NaN,10000.0
3,ACR103,2003,NaN,NaN,8072.0,NaN,2500.0
4,ACR103,2004,NaN,NaN,5848.0,NaN,32.0


In [149]:
cred_group_iss = cred_piv_iss.groupby(['project_id']).agg({'issuance_date':'min',
                                               'buffer_contribution':'sum', 
                                               'buffer_subtraction':'sum', 
                                               'issuance':'sum',
                                               'retirement_cancellation':'sum'}).reset_index()
cred_group_vin = cred_piv_vin.groupby(['project_id']).agg({'vintage_year':'min',
                                               'buffer_contribution':'sum', 
                                               'buffer_subtraction':'sum', 
                                               'issuance':'sum',
                                               'retirement_cancellation':'sum'}).reset_index()

In [150]:
cred_group_iss.columns = ['Project ID', 'First Issuance Year', 'Total Buffer Contributions by Issuance Year', 'Total Buffer Subtractions by Issuance Year',
                      'Total Credits Issued by Issuance Year', 'Total Credits Retired by Issuance Year']
cred_group_vin.columns = ['Project ID', 'First Vintage Year', 'Total Buffer Contributions by Vintage Year', 'Total Buffer Subtractions by Vintage Year',
                      'Total Credits Issued by Vintage Year', 'Total Credits Retired by Vintage Year']

In [151]:
cred_group_iss.head()

,Project ID,First Issuance Year,Total Buffer Contributions by Issuance Year,Total Buffer Subtractions by Issuance Year,Total Credits Issued by Issuance Year,Total Credits Retired by Issuance Year
0,ACR102,2005,0.0,0.0,7984006.0,17666.0
1,ACR103,2005,0.0,0.0,44202.0,28636.0
2,ACR105,2012,1102376.0,1102376.0,2572210.0,2572210.0
3,ACR106,2007,0.0,0.0,138492.0,109452.0
4,ACR107,2008,0.0,0.0,1138.0,127.0


In [156]:
pivot_iss_iss = cred_piv_iss.pivot(index='project_id', columns='issuance_date', values='issuance').reset_index()
pivot_iss_iss = pivot_iss_iss.drop(columns=["0"])
pivot_iss_iss.columns = [pivot_iss_iss.columns[0]] + [f"{col} Issuance by Issuance Year" for col in pivot_iss_iss.columns[1:]]
pivot_iss_iss.columns.values[0] = "Project ID"
pivot_iss_iss.fillna(0, inplace=True)
pivot_vin_iss = cred_piv_vin.pivot(index='project_id', columns='vintage_year', values='issuance').reset_index()
pivot_vin_iss = pivot_vin_iss.drop(columns=["0"])
pivot_vin_iss.columns = [pivot_vin_iss.columns[0]] + [f"{col} Issuance by Vintage Date" for col in pivot_vin_iss.columns[1:]]
pivot_vin_iss.columns.values[0] = "Project ID"
pivot_vin_iss.fillna(0, inplace=True)

In [157]:
pivot_vin_iss.head()

,Project ID,1996 Issuance by Vintage Date,1997 Issuance by Vintage Date,1998 Issuance by Vintage Date,1999 Issuance by Vintage Date,2000 Issuance by Vintage Date,2001 Issuance by Vintage Date,2002 Issuance by Vintage Date,2003 Issuance by Vintage Date,2004 Issuance by Vintage Date,2005 Issuance by Vintage Date,2006 Issuance by Vintage Date,2007 Issuance by Vintage Date,2008 Issuance by Vintage Date,2009 Issuance by Vintage Date,2010 Issuance by Vintage Date,2011 Issuance by Vintage Date,2012 Issuance by Vintage Date,2013 Issuance by Vintage Date,2014 Issuance by Vintage Date,2015 Issuance by Vintage Date,2016 Issuance by Vintage Date,2017 Issuance by Vintage Date,2018 Issuance by Vintage Date,2019 Issuance by Vintage Date,2020 Issuance by Vintage Date,2021 Issuance by Vintage Date,2022 Issuance by Vintage Date,2023 Issuance by Vintage Date,2024 Issuance by Vintage Date,2025 Issuance by Vintage Date,2026 Issuance by Vintage Date,2027 Issuance by Vintage Date,2028 Issuance by Vintage Date,2029 Issuance by Vintage Date,2030 Issuance by Vintage Date,2031 Issuance by Vintage Date,2032 Issuance by Vintage Date,2033 Issuance by Vintage Date,2034 Issuance by Vintage Date,2035 Issuance by Vintage Date,2036 Issuance by Vintage Date,2037 Issuance by Vintage Date,2038 Issuance by Vintage Date,2039 Issuance by Vintage Date,2040 Issuance by Vintage Date,2041 Issuance by Vintage Date,2042 Issuance by Vintage Date,2043 Issuance by Vintage Date,2044 Issuance by Vintage Date,2045 Issuance by Vintage Date,2046 Issuance by Vintage Date,2047 Issuance by Vintage Date,2048 Issuance by Vintage Date,2049 Issuance by Vintage Date,2050 Issuance by Vintage Date,2051 Issuance by Vintage Date,2052 Issuance by Vintage Date,2053 Issuance by Vintage Date,2054 Issuance by Vintage Date,2055 Issuance by Vintage Date,2056 Issuance by Vintage Date,2057 Issuance by Vintage Date,2058 Issuance by Vintage Date,2059 Issuance by Vintage Date,2060 Issuance by Vintage Date,2061 Issuance by Vintage Date
0,ACR102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1898382.0,3004613.0,3081011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ACR103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8072.0,5848.0,4409.0,3632.0,7998.0,13210.0,1033.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ACR105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2572210.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ACR106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95330.0,43162.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ACR107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,316.0,352.0,342.0,128.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [158]:
pivot_iss_rc= cred_piv_iss.pivot(index='project_id', columns='issuance_date', values='retirement_cancellation').reset_index()
pivot_iss_rc = pivot_iss_rc.drop(columns=["0"])
pivot_iss_rc.columns = [pivot_iss_rc.columns[0]] + [f"{col} Retirement/Cancellation by Issuance Year" for col in pivot_iss_rc.columns[1:]]
pivot_iss_rc.columns.values[0] = "Project ID"
pivot_iss_rc.fillna(0, inplace=True)
pivot_vin_rc= cred_piv_vin.pivot(index='project_id', columns='vintage_year', values='retirement_cancellation').reset_index()
pivot_vin_rc = pivot_vin_rc.drop(columns=["0"])
pivot_vin_rc.columns = [pivot_vin_rc.columns[0]] + [f"{col} Retirement/Cancellation by Vintage Year" for col in pivot_vin_rc.columns[1:]]
pivot_vin_rc.columns.values[0] = "Project ID"
pivot_vin_rc.fillna(0, inplace=True)

In [159]:
pivot_vin_rc.head()

,Project ID,1996 Retirement/Cancellation by Vintage Year,1997 Retirement/Cancellation by Vintage Year,1998 Retirement/Cancellation by Vintage Year,1999 Retirement/Cancellation by Vintage Year,2000 Retirement/Cancellation by Vintage Year,2001 Retirement/Cancellation by Vintage Year,2002 Retirement/Cancellation by Vintage Year,2003 Retirement/Cancellation by Vintage Year,2004 Retirement/Cancellation by Vintage Year,2005 Retirement/Cancellation by Vintage Year,2006 Retirement/Cancellation by Vintage Year,2007 Retirement/Cancellation by Vintage Year,2008 Retirement/Cancellation by Vintage Year,2009 Retirement/Cancellation by Vintage Year,2010 Retirement/Cancellation by Vintage Year,2011 Retirement/Cancellation by Vintage Year,2012 Retirement/Cancellation by Vintage Year,2013 Retirement/Cancellation by Vintage Year,2014 Retirement/Cancellation by Vintage Year,2015 Retirement/Cancellation by Vintage Year,2016 Retirement/Cancellation by Vintage Year,2017 Retirement/Cancellation by Vintage Year,2018 Retirement/Cancellation by Vintage Year,2019 Retirement/Cancellation by Vintage Year,2020 Retirement/Cancellation by Vintage Year,2021 Retirement/Cancellation by Vintage Year,2022 Retirement/Cancellation by Vintage Year,2023 Retirement/Cancellation by Vintage Year,2024 Retirement/Cancellation by Vintage Year,2025 Retirement/Cancellation by Vintage Year,2026 Retirement/Cancellation by Vintage Year,2027 Retirement/Cancellation by Vintage Year,2028 Retirement/Cancellation by Vintage Year,2029 Retirement/Cancellation by Vintage Year,2030 Retirement/Cancellation by Vintage Year,2031 Retirement/Cancellation by Vintage Year,2032 Retirement/Cancellation by Vintage Year,2033 Retirement/Cancellation by Vintage Year,2034 Retirement/Cancellation by Vintage Year,2035 Retirement/Cancellation by Vintage Year,2036 Retirement/Cancellation by Vintage Year,2037 Retirement/Cancellation by Vintage Year,2038 Retirement/Cancellation by Vintage Year,2039 Retirement/Cancellation by Vintage Year,2040 Retirement/Cancellation by Vintage Year,2041 Retirement/Cancellation by Vintage Year,2042 Retirement/Cancellation by Vintage Year,2043 Retirement/Cancellation by Vintage Year,2044 Retirement/Cancellation by Vintage Year,2045 Retirement/Cancellation by Vintage Year,2046 Retirement/Cancellation by Vintage Year,2047 Retirement/Cancellation by Vintage Year,2048 Retirement/Cancellation by Vintage Year,2049 Retirement/Cancellation by Vintage Year,2050 Retirement/Cancellation by Vintage Year,2051 Retirement/Cancellation by Vintage Year,2052 Retirement/Cancellation by Vintage Year,2053 Retirement/Cancellation by Vintage Year,2054 Retirement/Cancellation by Vintage Year,2055 Retirement/Cancellation by Vintage Year,2056 Retirement/Cancellation by Vintage Year,2057 Retirement/Cancellation by Vintage Year,2058 Retirement/Cancellation by Vintage Year,2059 Retirement/Cancellation by Vintage Year,2060 Retirement/Cancellation by Vintage Year,2061 Retirement/Cancellation by Vintage Year
0,ACR102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7666.0,0.0,10000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ACR103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2500.0,32.0,231.0,3632.0,7998.0,13210.0,1033.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ACR105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2572210.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ACR106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95330.0,14122.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [161]:
# Function to list all DataFrames in the current environment
def list_dataframes():
    return [var_name for var_name, var_val in globals().items() if isinstance(var_val, pd.DataFrame)]

# List all DataFrames
dataframe_list = list_dataframes()
print("DataFrames in the current environment:")
print(dataframe_list)

DataFrames in the current environment:
['_', '__', '___', 'cred', 'proj', '_5', 'proj1', 'proj2', '_34', '_55', '_56', 'cred_group_issuanceDate', 'cred_group_vintageDate', 'cred_piv_iss', 'cred_piv_vin', '_123', '_124', '_136', '_140', '_141', 'cred_group_iss', '_148', 'cred_group_vin', '_151', 'pivot_iss_iss', 'pivot_vin_iss', '_153', '_155', '_157', 'pivot_iss_rc', 'pivot_vin_rc', '_159']


In [162]:
merged_df = proj1.merge(cred_group_iss, on='Project ID', how='outer') \
                  .merge(cred_group_vin, on='Project ID', how='outer') \
                  .merge(pivot_iss_iss, on='Project ID', how='outer') \
                  .merge(pivot_vin_iss, on='Project ID', how='outer') \
                  .merge(pivot_iss_rc, on='Project ID', how='outer') \
                  .merge(pivot_vin_rc, on='Project ID', how='outer') \
                  .merge(proj2, on='Project ID', how='outer')

In [163]:
merged_df.head()

,Project ID,Project Name,Voluntary Registry,ARB/WA Project,Voluntary Status,Scope,Type,Type Rule,Type ML,Confidence Level,Reduction/Removal,Methodology/Protocol,Methodology Version,Region,Country,State,Project Site Location,Project Developer,Project Owner,First Issuance Year,Total Buffer Contributions by Issuance Year,Total Buffer Subtractions by Issuance Year,Total Credits Issued by Issuance Year,Total Credits Retired by Issuance Year,First Vintage Year,Total Buffer Contributions by Vintage Year,Total Buffer Subtractions by Vintage Year,Total Credits Issued by Vintage Year,Total Credits Retired by Vintage Year,2002 Issuance by Issuance Year,2004 Issuance by Issuance Year,2005 Issuance by Issuance Year,2006 Issuance by Issuance Year,2007 Issuance by Issuance Year,2008 Issuance by Issuance Year,2009 Issuance by Issuance Year,2010 Issuance by Issuance Year,2011 Issuance by Issuance Year,2012 Issuance by Issuance Year,2013 Issuance by Issuance Year,2014 Issuance by Issuance Year,2015 Issuance by Issuance Year,2016 Issuance by Issuance Year,2017 Issuance by Issuance Year,2018 Issuance by Issuance Year,2019 Issuance by Issuance Year,2020 Issuance by Issuance Year,2021 Issuance by Issuance Year,2022 Issuance by Issuance Year,2023 Issuance by Issuance Year,2024 Issuance by Issuance Year,1996 Issuance by Vintage Date,1997 Issuance by Vintage Date,1998 Issuance by Vintage Date,1999 Issuance by Vintage Date,2000 Issuance by Vintage Date,2001 Issuance by Vintage Date,2002 Issuance by Vintage Date,2003 Issuance by Vintage Date,2004 Issuance by Vintage Date,2005 Issuance by Vintage Date,2006 Issuance by Vintage Date,2007 Issuance by Vintage Date,2008 Issuance by Vintage Date,2009 Issuance by Vintage Date,2010 Issuance by Vintage Date,2011 Issuance by Vintage Date,2012 Issuance by Vintage Date,2013 Issuance by Vintage Date,2014 Issuance by Vintage Date,2015 Issuance by Vintage Date,2016 Issuance by Vintage Date,2017 Issuance by Vintage Date,2018 Issuance by Vintage Date,2019 Issuance by Vintage Date,2020 Issuance by Vintage Date,2021 Issuance by Vintage Date,2022 Issuance by Vintage Date,2023 Issuance by Vintage Date,2024 Issuance by Vintage Date,2025 Issuance by Vintage Date,2026 Issuance by Vintage Date,2027 Issuance by Vintage Date,2028 Issuance by Vintage Date,2029 Issuance by Vintage Date,2030 Issuance by Vintage Date,2031 Issuance by Vintage Date,2032 Issuance by Vintage Date,2033 Issuance by Vintage Date,2034 Issuance by Vintage Date,2035 Issuance by Vintage Date,2036 Issuance by Vintage Date,2037 Issuance by Vintage Date,2038 Issuance by Vintage Date,2039 Issuance by Vintage Date,2040 Issuance by Vintage Date,2041 Issuance by Vintage Date,2042 Issuance by Vintage Date,2043 Issuance by Vintage Date,2044 Issuance by Vintage Date,2045 Issuance by Vintage Date,2046 Issuance by Vintage Date,2047 Issuance by Vintage Date,2048 Issuance by Vintage Date,2049 Issuance by Vintage Date,2050 Issuance by Vintage Date,2051 Issuance by Vintage Date,2052 Issuance by Vintage Date,2053 Issuance by Vintage Date,2054 Issuance by Vintage Date,2055 Issuance by Vintage Date,2056 Issuance by Vintage Date,2057 Issuance by Vintage Date,2058 Issuance by Vintage Date,2059 Issuance by Vintage Date,2060 Issuance by Vintage Date,2061 Issuance by Vintage Date,2002 Retirement/Cancellation by Issuance Year,2004 Retirement/Cancellation by Issuance Year,2005 Retirement/Cancellation by Issuance Year,2006 Retirement/Cancellation by Issuance Year,2007 Retirement/Cancellation by Issuance Year,2008 Retirement/Cancellation by Issuance Year,2009 Retirement/Cancellation by Issuance Year,2010 Retirement/Cancellation by Issuance Year,2011 Retirement/Cancellation by Issuance Year,2012 Retirement/Cancellation by Issuance Year,2013 Retirement/Cancellation by Issuance Year,2014 Retirement/Cancellation by Issuance Year,2015 Retirement/Cancellation by Issuance Year,2016 Retirement/Cancellation by Issuance Year,2017 Retirement/Cancellation by Issuance Year,2018 Retirement/Cancellation 

In [164]:
merged_df['Registry/ARB/WA'] = merged_df['Voluntary Registry']
merged_df.loc[merged_df['ARB/WA Project'].isin(['ARB Compliance', 'ARB Early Action']), 'Registry/ARB/WA'] = 'ARB'
merged_df.loc[merged_df['ARB/WA Project'].isin(['WA Compliance', 'WA Early Action']), 'Registry/ARB/WA'] = 'WA'

In [168]:
merged_df[['Total Buffer Contributions by Issuance Year','Total Buffer Subtractions by Issuance Year','Total Credits Issued by Issuance Year','Total Credits Retired by Issuance Year']] = merged_df[['Total Buffer Contributions by Issuance Year','Total Buffer Subtractions by Issuance Year','Total Credits Issued by Issuance Year','Total Credits Retired by Issuance Year']].fillna(0).astype(int)
merged_df['Total Credits Retired by Issuance Year'] = merged_df['Total Credits Retired by Issuance Year'].abs()
merged_df[['Total Buffer Contributions by Vintage Year','Total Buffer Subtractions by Vintage Year','Total Credits Issued by Vintage Year','Total Credits Retired by Vintage Year']] = merged_df[['Total Buffer Contributions by Vintage Year','Total Buffer Subtractions by Vintage Year','Total Credits Issued by Vintage Year','Total Credits Retired by Vintage Year']].fillna(0).astype(int)
merged_df['Total Credits Retired by Vintage Year'] = merged_df['Total Credits Retired by Vintage Year'].abs()

In [175]:
merged_df.columns[-20:]

Index(['Authorized Project Designee', 'Verifier',
       'Estimated Annual Emission Reductions', 'ARB ID', 'ARB Project Type',
       'ARB Project Status', 'WA ID', 'WA Project Type', 'POA ID',
       'Project Listed', 'Project Registered', 'Sustainability Certification',
       'Project Type From the Registry', 'Registry Documents',
       'Project Website', 'Notes From Registry', 'Notes from BCTP',
       'Project Description', 'Added to DB Version', 'Registry/ARB/WA'],
      dtype='object')

In [ ]:
merged_df = merged_df.drop(columns=['col1', 'col2'])

In [169]:
merged_df.to_csv("Project_Tab.csv", index=False)

In [ ]:
cred_piv.columns = ['Project ID', 'Year', 'Total Buffer Contributions by Year','Total Buffer Subtractions by Year','Total Credits Issued by Year','Per','Total Credits Retired by Year']

In [ ]:
merged_df2 =  proj1.merge(cred_piv, on='Project ID', how ='outer') \
                    .merge(prev, on='Project ID', how ='outer') \
                    .merge(pivot1, on='Project ID', how='outer')

merged_df2['Registry/ARB/WA'] = merged_df2['Voluntary Registry']
merged_df2.loc[merged_df2['ARB/WA Project'].isin(['ARB Compliance', 'ARB Early Action']), 'Registry/ARB/WA'] = 'ARB'
merged_df2.loc[merged_df2['ARB/WA Project'].isin(['WA Compliance', 'WA Early Action']), 'Registry/ARB/WA'] = 'WA'

merged_df2[['Total Buffer Contributions','Total Buffer Subtractions','Total Credits Issued','Total Credits Retired']] = merged_df2[['Total Buffer Contributions','Total Buffer Subtractions','Total Credits Issued','Total Credits Retired']].fillna(0).astype(int)
merged_df2['Total Credits Retired'] = merged_df2['Total Credits Retired'].abs()

In [ ]:
merged_df2.head()

In [ ]:
merged_df.to_csv("../../streamlit_test/data/Project_Tab_Pivoted.csv", index=False)